In [172]:
import os
import numpy as np
import pandas as pd
import time
from pandas import DataFrame, Series
import datetime
from matplotlib import pyplot as plt
%matplotlib inline
import random
import math as math
import matplotlib.path as mplPath

In [173]:
os.chdir("/Users/EricYi/Desktop/Code/datathon_data")

In [174]:
def rand_samp_csv(filename, divisor=1000):
    n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
    s = int(n/divisor) #desired sample size
    skip = sorted(random.sample(xrange(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
    df = pd.read_csv(filename, skiprows=skip)
    print "done"
    return df

# non-trip data
geographic = pd.read_csv('geographic.csv')
demographics = pd.read_csv("demographics.csv")
zones = pd.read_csv("zones.csv")

# uber trip data
uber_2014 = rand_samp_csv("uber_trips_2014.csv")
uber_2015 = rand_samp_csv("uber_trips_2015.csv")

# green trip data
green_trips = rand_samp_csv("green_trips.csv")

# yellow trip data
yellow_trips_2014Q2 = rand_samp_csv("yellow_trips_2014Q2.csv")
yellow_trips_2014Q3 = rand_samp_csv("yellow_trips_2014Q3.csv")
yellow_trips_2015Q1 = rand_samp_csv("yellow_trips_2015Q1.csv")
yellow_trips_2015Q2 = rand_samp_csv("yellow_trips_2015Q2.csv")

# other data
other_fhv_trips = rand_samp_csv("other_fhv_trips.csv")

done
done
done
done
done
done
done
done


In [175]:
# merging trips with zones based on Taxi Zones to obtain NTA codes from zones
uber_zones_2015 = uber_2015.merge(zones, left_on='pickup_location_id', right_on='location_id', how='outer')

In [176]:
# merging trips and zones with demographics based on NTA codes to obtain demographics
final_uber_2015 = uber_zones_2015.merge(demographics, left_on='nta_code', right_on='nta_code', how='outer')

In [177]:
def to_Dates(sample):
    # converts datetimes to dates
    sample['pickup_date'] = pd.DatetimeIndex(
        sample['pickup_datetime'].apply(pd.to_datetime)).date
    print "datetime conversion done"
    sample['count'] = 1
    return sample

In [178]:
# Compute NTA Paths
NTA_names = geographic.loc[[1]]
NTA_paths = []
for x in NTA_names:
    geo_x_filtered = filter(lambda k: not math.isnan(k), geographic[x])
    num_vertices = min(2000, len(geo_x_filtered))
    polygon = map(lambda i: [geo_x_filtered[2*i], geo_x_filtered[2*i+1]], xrange(0, num_vertices/2))
    NTA_paths.append((x, mplPath.Path(np.array(polygon))))

In [179]:
def find_districts(sample, rows):
    sLength = len(sample['pickup_datetime'])
    sample['nta_code'] = Series(np.random.randn(sLength))

    start_time = time.time()
    districts = []
    counter = 0
    for index, row in sample[0:rows].iterrows():
        counter += 1000
        latitude = row['pickup_latitude']
        longitude = row['pickup_longitude']

        loc = "None"
        if (len(districts) % 1000 == 0):
            print "i = ", counter
        for (district, path) in NTA_paths:
            if path.contains_point((longitude, latitude)):
                loc = district
                break
        districts.append(loc)
    time_taken = time.time() - start_time
    print 'time taken: ', time_taken
    return districts

In [180]:
def insertDistricts(sample, districts):
    sample["nta_code"] = districts
    sample["count"] = 1
    print "districts done"
    return sample

In [181]:
def merge_withDemo(sample):
    return sample.merge(demographics, left_on='nta_code', right_on='nta_code', how='outer')

In [182]:
def output_stuff(sample, filenames):
    use = sample.groupby(by=["nta_code", "pickup_date"])
    final = use.sum()
    final.to_csv(filenames[0])
    
    #use2 = sample.groupby(by=["nta_code"])
    #final2 = use2.sum()
    #final2.to_csv(filenames[1])
    return final

In [183]:
# uber_2015,other_fhv_trips are the frames without coordinates
df_loc_list = [uber_2014, green_trips, yellow_trips_2014Q2,
 yellow_trips_2014Q3, yellow_trips_2015Q1, yellow_trips_2015Q2]
for i in xrange(len(df_loc_list)):
    sample = df_loc_list[i]
    df_loc_list[i] = merge_withDemo(insertDistricts(to_Dates(sample), find_districts(sample, len(sample))))

#output_stuff(final_uber_2015, ["IMBALANCED_uber_trips_2015_days.csv", "IMBALANCED_uber_trips_2015_year.csv"])

datetime conversion done
i =  1000
i =  1001000
i =  2001000
i =  3001000
i =  4001000
time taken:  6.67237377167
districts done
datetime conversion done
i =  1000
i =  1001000
i =  2001000
i =  3001000
i =  4001000
i =  5001000
i =  6001000
i =  7001000
i =  8001000
i =  9001000
i =  10001000
i =  11001000
i =  12001000
i =  13001000
i =  14001000
i =  15001000
i =  16001000
i =  17001000
time taken:  21.8658368587
districts done
datetime conversion done
i =  1000
i =  1001000
i =  2001000
i =  3001000
i =  4001000
i =  5001000
i =  6001000
i =  7001000
i =  8001000
i =  9001000
i =  10001000
i =  11001000
i =  12001000
i =  13001000
i =  14001000
i =  15001000
i =  16001000
i =  17001000
i =  18001000
i =  19001000
i =  20001000
i =  21001000
i =  22001000
i =  23001000
i =  24001000
i =  25001000
i =  26001000
i =  27001000
i =  28001000
i =  29001000
i =  30001000
i =  31001000
i =  32001000
i =  33001000
i =  34001000
i =  35001000
i =  36001000
i =  37001000
i =  38001000
i =  39

In [184]:
sample = df_loc_list[0]

def merger(sample):
    use = sample[["service", "nta_code", "pickup_date", "count", "population", "median_age", "households",
              "median_income", "people_per_acre"]].groupby(by=["service", "nta_code", "pickup_date"])
    final = use.agg({'count': 'sum', 'population': 'mean', 'median_age': 'median',
                'households': 'mean', 'median_income': 'median', 'people_per_acre': 'mean'})
    return final

In [188]:
# this is so ugly

#df_loc_list = [uber_2014, green_trips, yellow_trips_2014Q2,
# yellow_trips_2014Q3, yellow_trips_2015Q1, yellow_trips_2015Q2]
#uber_zones_2015

uber_2014['service'] = 'uber'
uber_2014F = merger(merge_withDemo(uber_2014))
uber_2014F.columns=["count","avg_ppl_per_acre", "median_median_income", "median_median_age", "avg_households",
                  "avg_population"]

uber_zones_2015s = uber_zones_2015
uber_zones_2015s['service'] = "uber"
uber_2015F = merger(merge_withDemo(to_Dates(uber_zones_2015)))
uber_2015F.columns=["count","avg_ppl_per_acre", "median_median_income", "median_median_age", "avg_households",
                  "avg_population"]

green_trips["service"] = "green"
green_tripsF = merger(merge_withDemo(green_trips))
green_tripsF.columns=["count","avg_ppl_per_acre", "median_median_income", "median_median_age", "avg_households",
                  "avg_population"]

yellow_trips_2014Q2["service"] = "yellow"
yellow_trips_2014Q2F = merger(merge_withDemo(yellow_trips_2014Q2))
yellow_trips_2014Q2F.columns=["count","avg_ppl_per_acre", "median_median_income", "median_median_age", "avg_households",
                  "avg_population"]

yellow_trips_2014Q3["service"] = "yellow"
yellow_trips_2014Q3F = merger(merge_withDemo(yellow_trips_2014Q3))
yellow_trips_2014Q3F.columns=["count","avg_ppl_per_acre", "median_median_income", "median_median_age", "avg_households",
                  "avg_population"]

yellow_trips_2015Q1["service"] = "yellow"
yellow_trips_2015Q1F = merger(merge_withDemo(yellow_trips_2015Q1))
yellow_trips_2015Q1F.columns=["count","avg_ppl_per_acre", "median_median_income", "median_median_age", "avg_households",
                  "avg_population"]

yellow_trips_2015Q2["service"] = "yellow"
yellow_trips_2015Q2F = merger(merge_withDemo(yellow_trips_2015Q2))
yellow_trips_2015Q2F.columns=["count","avg_ppl_per_acre", "median_median_income", "median_median_age", "avg_households",
                  "avg_population"]


datetime conversion done


In [189]:
big_Frame = pd.concat([uber_2014F, uber_2015F, green_tripsF, yellow_trips_2014Q2F,
                       yellow_trips_2014Q3F, yellow_trips_2015Q1F, yellow_trips_2015Q2F])


In [190]:
big_Frame.to_csv("big_frame_final.csv")

In [ ]:
green_trips = rand_samp_csv("green_trips.csv")
green_trips_Af = merge_withDemo(insertDistricts(to_Dates(green_trips), find_districts(green_trips, len(green_trips))))

In [ ]:
green_trips_Af['service'] = 'green'
green_trips_f = merger(green_trips_Af)
green_trips_f

In [ ]:
uber_2014